In [1]:
import requests
import pandas as pd
import numpy as np
# import boto3
# import botocore
import time
from contextlib import closing
import csv
import string

In [2]:
test = pd.read_csv("oca_stat.csv")

In [2]:
link_base= "https://www.nycourts.gov/"
link_path = "LegacyPDFS/court-research/OCA-STAT-Act.csv"
link_url = link_base+link_path
link_url

'https://www.nycourts.gov/LegacyPDFS/court-research/OCA-STAT-Act.csv'

In [3]:
data_list = []
with closing(requests.get(link_url, stream=True)) as r:
    f = (line.decode('utf-8', 'ignore') for line in r.iter_lines())
    reader = csv.reader(f, delimiter=',')
    for row in reader:
        data_list.append(row)

In [4]:
oca_data = pd.DataFrame(data_list[1:], columns=data_list[0])
oca_data.dropna(axis = 0, how = 'any', inplace=True)

In [5]:
oca_data.columns = oca_data.columns.str.lower()
oca_data.columns = oca_data.columns.str.replace(" ", "_").str.replace(".", "_")

<ipython-input-5-e9adfcb8168f>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  oca_data.columns = oca_data.columns.str.replace(" ", "_").str.replace(".", "_")


In [6]:
oca_data.arraign_year = pd.to_numeric(oca_data.arraign_year, 
                                      errors='coerce') \
                            .fillna(0).astype(np.int64)
oca_data.arraign_month = pd.to_numeric(oca_data.arraign_month, 
                                      errors='coerce') \
                            .fillna(0).astype(np.int64)
oca_data.arrest_age = pd.to_numeric(oca_data.arrest_age, 
                                      errors='coerce')
oca_data.fines_imposed = pd.to_numeric(oca_data.fines_imposed, 
                                      errors='coerce')
oca_data.fees_imposed = pd.to_numeric(oca_data.fees_imposed, 
                                      errors='coerce')
oca_data.surcharges_imposed = pd.to_numeric(oca_data.surcharges_imposed, 
                                      errors='coerce')

In [7]:
# basic clean up and standardization 
oca_data = oca_data.applymap(lambda x:x.lower() if type(x) == str else x)
oca_data.article_section = oca_data.article_section.str.rstrip(".")
oca_data.article_section = oca_data.article_section.str.replace("\.\.", ".")
oca_data.drop("row_#",axis=1, inplace=True)
oca_data[["article_bxd", "section_bxd"]] = \
    oca_data.article_section.str.split(".", n=1,expand=True)

<ipython-input-7-3929d0c6ddbb>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  oca_data.article_section = oca_data.article_section.str.replace("\.\.", ".")


In [8]:
from oca_utils.oca_utils import *
oca_data = make_clean_precincts(oca_data)

fel_weights = [x for x in oca_data.weight.unique() if "f" in x]
fel_series = oca_data[oca_data.weight.isin(fel_weights)] \
    .article_bxd.apply(lambda x: make_felonies(x))
vio_series = oca_data[oca_data.law.isin(["vtl"])] \
    .article_bxd.apply(lambda x: make_violations(x))
bxd_category = pd.concat([fel_series, vio_series]) \
    .rename("category_bxd")
oca_data = oca_data.join(bxd_category)

/Users/jeffshamp/Documents/GitHub/OCA_data/oca_utils/oca_utils.py:97: FutureWarning: The default value of regex will change from True to False in a future version.
  oca_data.arresting_agency = oca_data.arresting_agency.str.replace(
/Users/jeffshamp/Documents/GitHub/OCA_data/oca_utils/oca_utils.py:106: FutureWarning: The default value of regex will change from True to False in a future version.
  oca_data.arresting_agency = oca_data.arresting_agency.str.replace(
/Users/jeffshamp/Documents/GitHub/OCA_data/oca_utils/oca_utils.py:112: FutureWarning: The default value of regex will change from True to False in a future version.
  oca_data.arresting_agency = oca_data.arresting_agency.str.replace(
/Users/jeffshamp/Documents/GitHub/OCA_data/oca_utils/oca_utils.py:114: FutureWarning: The default value of regex will change from True to False in a future version.
  oca_data.arresting_agency = oca_data.arresting_agency.str.replace(
/Users/jeffshamp/Documents/GitHub/OCA_data/oca_utils/oca_utils.py

In [9]:
weapons = ("|").join(["wea", "wpn", "wep"])
oca_data['weapon_charge_bxd'] = False
oca_data.loc[(oca_data.top_charge_at_arraignment.str.contains(weapons)) &
    (oca_data.weight != "v"), 'weapon_charge_bxd'] = True

sex_charge = ("|").join(["sex", "sx"])
oca_data["sex_charge_bxd"] = False
oca_data.loc[(oca_data.top_charge_at_arraignment.str.contains(sex_charge)),
         "sex_charge_bxd"] = True

In [10]:
for col in oca_data:
    if oca_data[col].dtype == "O":
        oca_data[col] = oca_data[col].str.replace('[{}]'.format(string.punctuation), ' ')

<ipython-input-10-8ef1614ee11c>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  oca_data[col] = oca_data[col].str.replace('[{}]'.format(string.punctuation), ' ')


In [11]:
#import unidecode
# def remove_accents(row):
#     return unicode(row, "utf-8")
# oca_data.top_charge_at_arraignment = \
#     oca_data.top_charge_at_arraignment.apply(lambda x: remove_accents(x))

In [12]:
oca_data[oca_data.article_section.str.contains("511")][['top_charge_at_arraignment', 'article_section', 'law', 'severity', 'weight']]

,top_charge_at_arraignment,article_section,law,severity,weight
3,aggravated unlic oper mv 3rd,0511,vtl,misdemeanor,um
8,aggravated unlic oper veh 3rd,0511,vtl,misdemeanor,um
19,aggravated unlic oper veh 3rd,0511,vtl,misdemeanor,um
50,agg unlic oper 2nd 3 suspensns,0511,vtl,misdemeanor,um
53,aggravated unlic oper veh 3rd,0511,vtl,misdemeanor,um
...,...,...,...,...,...
215004,agg unlic oper 1 10 more susp,511 e2,vtl,felony,ef
215251,agg unlic oper 1 alcohol,511 e1,vtl,felony,ef
215251,agg unlic oper 1 alcohol,511 e1,vtl,felony,ef
215399,agg unlic oper 1 alcohol,511 e1,vtl,felony,ef


In [18]:
write_csv = False
if write_csv:
    oca_data.to_csv("codec_test.csv", encoding='utf-8')

In [13]:
oca_data[(oca_data.article_bxd == "230" ) |
        (oca_data.article_bxd == "225")] \
        .groupby(["court"]).count()['region'].sort_values(ascending=False)

court
bronx criminal court              160
queens criminal court              64
kings criminal court               40
new york criminal court            36
bronx supreme court                15
suffolk 1st district court         13
nassau district court              11
richmond criminal court            10
queens supreme court                7
colonie t j                         5
syracuse city court                 5
kings supreme court                 4
new york supreme court              4
lancaster t j                       3
clifton park t j                    3
steuben county court                3
fulton city court                   2
peekskill city court                2
middletown city court               2
mount vernon city court             2
nassau county court                 2
utica city court                    2
buffalo city court                  2
hornellsville t j                   2
southampton t j                     1
suffolk county court                1
spring

In [16]:
oca_data.groupby("arraign_month").count()["court"]

arraign_month
1     20202
2     19004
3     27342
4     24981
11    26157
12    20992
Name: court, dtype: int64

## Slated for deletion

In [4]:
test.columns

Index(['Unnamed: 0', 'court_type', 'region', 'district', 'county', 'court',
       'arresting_agency', 'arrest_type', 'arraign_year', 'arraign_month',
       'top_charge_at_arraignment', 'severity', 'weight', 'law',
       'article_section', 'attempt_flag', 'gender', 'ethnicity', 'race',
       'arrest_age', 'docket_status', 'disposition_type', 'disposition_detail',
       'dismissal_reason', 'most_severe_sentence', 'fines_imposed',
       'fees_imposed', 'surcharges_imposed', 'article_bxd', 'section_bxd',
       'category_bxd', 'weapon_charge_bxd', 'sex_charge_bxd'],
      dtype='object')

In [20]:
test = test[(test.article_section.isin(["125.25", "120.10"]))]
groups = test.groupby(["top_charge_at_arraignment", "disposition_type",
              "disposition_detail", "dismissal_reason", "most_severe_sentence"]) \
            .count()['court_type'] \
            .sort_values(ascending=False) \
            .reset_index()
groups

,top_charge_at_arraignment,disposition_type,disposition_detail,dismissal_reason,most_severe_sentence,court_type
0,murder: intention,,,,,532
1,murder: intention,gj/trans,transfer to superior court,,,238
2,aslt:w/int caus ser inj w/weap,,,,,205
3,aslt w/int caus ser inj w/weap,gj/trans,transfer to superior court,,,197
4,aslt w/int caus ser inj w/weap,,,,,142
...,...,...,...,...,...,...
88,murder - 2nd deg,dismissed,dismissed and superseded,dismissed and superseded,,1
89,murder-2: during specified fel,dismissed,dismissed,interest/furtherance of justice (cpl 170.30 (1...,,1
90,murder-2:deprav-kill person<11,gj/trans,transfer to superior court,,,1
91,murder-2nd deg,,,,,1


In [ ]:
# if is_downloadable(link_url, mod_list):
#     r = requests.get(link_url)
#     last_mod = r.headers['Last-Modified']
#     mod_list.append(last_mod)
#     oca_data = pd.read_csv(io.StringIO(r.text))
# else: print("File not downloadable or exists already.")

In [ ]:
# if mod_list:
#     with open("OCA_last_modified.txt", "wb") as fp:
#         pickle.dump(mod_list, fp)

In [ ]:
# Re-scrape - Takes a while so use cautiously
re_load = False
if re_load:
    r = requests.get(link_url)
    oca_data = pd.read_csv(io.StringIO(r.text))

In [ ]:
oca_main = oca_data.copy()

In [ ]:
# Restore from copy
quick_load = False
if quick_load:
    oca_data = oca_main

In [ ]:
test = oca_data.drop("top_charge_at_arraignment", axis=1)
for col in test:
    if test[col].dtype == "O":
        print(col)
        test[col] = test[col].str.encode("utf-8")

In [ ]:
oca_data.top_charge_at_arraignment = \
    oca_data.top_charge_at_arraignment.replace({' +':' '}, regex=True)
type(oca_data.top_charge_at_arraignment)

In [ ]:
oca_data.arresting_agency.str.encode('utf-8')

## BXD S3 Upload

In [ ]:
tgt_bucket = 'oca-data-test'
tgt_csv = 'oca_data_clean_final'
s3 = boto3.resource('s3')

today_str = time.strftime("%Y-%m-%d")
#download_folder = '/home/ec2-user/gcloud/docs/'

f = open(tgt_csv+'.csv', 'rb')
s3.Bucket(tgt_bucket).put_object(
                        Key='extract_'+today_str+'/'+tgt_csv+'.csv',
                        ServerSideEncryption='aws:kms',
                        StorageClass='STANDARD_IA',
                        Body=f)